# Create Single household load profile

Power converted to kW

In [1]:
%%capture
!pip install plotly==5.9.0

In [2]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/data/data_testing/
%ls

In [4]:
#Moving to current directory
current_folder = ''

input_folder = current_folder + 'raw_datasets/'
output_folder = current_folder + 'scenario_datasets/'

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px

In [6]:
household_data = pd.read_csv(input_folder + 'consumer-00000002_glimpse_TEST.csv')

household_data['time'] = pd.to_datetime(household_data['time'])
household_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14401 entries, 0 to 14400
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  14401 non-null  int64         
 1   time        14401 non-null  datetime64[ns]
 2   energy      14401 non-null  int64         
 3   power       14401 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 450.2 KB


In [7]:
household_data = household_data.drop(['energy'], axis=1)
household_data.tail()

,Unnamed: 0,time,power
14396,14397,2017-06-30 23:48:00,401459
14397,14398,2017-06-30 23:51:00,398947
14398,14399,2017-06-30 23:54:00,398824
14399,14400,2017-06-30 23:57:00,400042
14400,14401,2017-07-01 00:00:00,344167


In [8]:
#Resampling time

time_sampled = household_data.resample('15min', on='time').time.sum
household_data = household_data.resample('15min', on='time').agg({'time':'min', 'power':'mean'})
household_data = household_data.drop(['time'], axis=1)
household_data = household_data.loc['2017-01-01 00:00:00':'2018-01-01 00:00:00']
household_data[household_data["power"]==""] = np.NaN
household_data.power = household_data.power.ffill()
#Convertion from mW to kW
household_data['power'] = household_data['power']/1000000
household_data.head()

,power
time,
2017-06-01 00:00:00,0.403179
2017-06-01 00:15:00,0.405153
2017-06-01 00:30:00,0.537645
2017-06-01 00:45:00,0.779943
2017-06-01 01:00:00,0.732574


In [9]:
household_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2881 entries, 2017-06-01 00:00:00 to 2017-07-01 00:00:00
Freq: 15T
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   power   2881 non-null   float64
dtypes: float64(1)
memory usage: 45.0 KB


In [10]:
#Save and plot data
#household_csv = household_data.to_csv(output_folder + 'household_load_profile.csv', index = True)
px.line(household_data["power"], title=('Multiple household power consuption(4)')).update_layout(yaxis_title="kW", xaxis_rangeslider_visible=True, xaxis_range=["2017-06-01 00:00:00", "2017-07-01 00:00:00"])